In [1]:
# Import dependencies
import os

os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from src.utils.gpu_config import configure_gpu, optimize_tensorflow

configure_gpu()
optimize_tensorflow()

2025-05-21 23:51:12.502314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747864272.603078    4304 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747864272.632266    4304 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747864272.871244    4304 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747864272.871251    4304 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747864272.871253    4304 computation_placer.cc:177] computation placer alr


GPU configuration successful!
Device: /physical_device:GPU:0
Memory limit: 18.0 GB

TensorFlow optimizations status:
- CPU Threads: 16 intra-op, 9 inter-op (32 cores total)
- XLA JIT: enabled (autoclustering)
- Synchronous Execution: enabled

GPU Device Details:
- Device: GPU /physical_device:GPU:0
- Memory limit: 18.0 GB


I0000 00:00:1747864276.441032    4304 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1747864276.441876    4304 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19327352832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryFocalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, TF_SHUFFLE_SIZE, TF_BATCH_SIZE, TF_MAX_EPOCHS
from src.model.tensorflow_utils import load_dataset, optimize_dataset, count_dataset_size, get_num_classes, get_metrics
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_early_stopping, filter_no_finding_class
from src.model.tensorflow_utils import start_or_resume_training, apply_augmentation_to_dataset, calculate_class_weights, oversample_minority_classes
from src.model.densnet.tensorflow_dense_net_121 import build_densenet121

# Input Data
initial_epoch   = None
resume_training = False
checkpoint_path = None
set_name        = "DenseNetV3"
model_name      = f"{set_name}_2"
num_classes     = get_num_classes()

train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)

train_ds = filter_no_finding_class(train_ds)
val_ds   = filter_no_finding_class(val_ds)

class_weights = calculate_class_weights(train_ds, num_classes)
train_ds      = oversample_minority_classes(train_ds, class_weights)
class_weights = calculate_class_weights(train_ds, num_classes)

In [3]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Disable Advanced Augmentations
probability = {
    "intensity_scaling": 0,
    "adaptive_histogram": 0,
    "zoom": 0,
    "cutout": 0
}

train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds, probability)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds   = optimize_dataset(val_ds, TF_BATCH_SIZE)

Counting samples: 104166 samples [00:43, 2399.01 samples/s] 
Counting samples: 15431 samples [00:14, 1044.26 samples/s]


In [4]:
# Setup Model Deps
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
metrics           = get_metrics()

# Setup compile arguments
loss           = BinaryFocalCrossentropy(gamma=2.0, from_logits=False, label_smoothing=0.10)
reduce_lr      = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)
early_stopping = setup_early_stopping()

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max", save_weights_only=False)

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max", save_weights_only=False)

In [5]:
# Model Training
model          = build_densenet121(num_classes)
optimizer      = Adam(learning_rate=1e-4)
compile_kwargs = {'optimizer': optimizer, 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    TF_MAX_EPOCHS,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, early_stopping], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-05-21 23:54:06 - INFO - 
=== Training Started ===

2025-05-21 23:54:06 - INFO - Batch Size: 128
2025-05-21 23:54:06 - INFO - Optimizer: Adam
2025-05-21 23:54:06 - INFO - 

2025-05-21 23:54:06 - INFO - 
=== Starting Epoch 1 ===



Epoch 1/100


I0000 00:00:1747864456.735550    4366 service.cc:152] XLA service 0x737f9c003020 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747864456.735567    4366 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
I0000 00:00:1747864456.754109    4366 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1747864456.835428    4366 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


813/813 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.9062 - auc: 0.5524 - f1_score: 0.0473 - loss: 1.8423 - precision: 0.1389 - recall: 0.0317  

2025-05-21 23:59:43 - INFO - 
=== Epoch 1 Summary ===
2025-05-21 23:59:43 - INFO - Time: 337.58s
2025-05-21 23:59:43 - INFO - Training   - accuracy: 0.9158 - auc: 0.5756 - f1_score: 0.0371 - loss: 1.5681 - precision: 0.1715 - recall: 0.0215 - learning_rate: 0.0001
2025-05-21 23:59:43 - INFO - Validation - accuracy: 0.9445 - auc: 0.6464 - f1_score: 0.0025 - loss: 1.0791 - precision: 0.2778 - recall: 0.0013
2025-05-21 23:59:43 - INFO - ==================================================



...

Epoch 64/100
813/813 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step - accuracy: 0.9611 - auc: 0.9697 - f1_score: 0.7110 - loss: 0.0301 - precision: 0.8258 - recall: 0.6400  

2025-05-22 04:12:32 - INFO - 
=== Epoch 64 Summary ===
2025-05-22 04:12:32 - INFO - Time: 239.41s
2025-05-22 04:12:32 - INFO - Training   - accuracy: 0.9616 - auc: 0.9703 - f1_score: 0.7129 - loss: 0.0299 - precision: 0.8256 - recall: 0.6433 - learning_rate: 0.0000
2025-05-22 04:12:32 - INFO - Validation - accuracy: 0.9341 - auc: 0.7575 - f1_score: 0.2450 - loss: 0.0865 - precision: 0.3414 - recall: 0.2041
2025-05-22 04:12:32 - INFO - ==================================================



813/813 ━━━━━━━━━━━━━━━━━━━━ 239s 295ms/step - accuracy: 0.9611 - auc: 0.9697 - f1_score: 0.7110 - loss: 0.0301 - precision: 0.8258 - recall: 0.6400 - val_accuracy: 0.9341 - val_auc: 0.7575 - val_f1_score: 0.2450 - val_loss: 0.0865 - val_precision: 0.3414 - val_recall: 0.2041 - learning_rate: 1.0000e-06


2025-05-22 04:12:32 - INFO - 
=== Training Completed! ===

2025-05-22 04:12:32 - INFO - Final Metrics: accuracy: 0.9616 - auc: 0.9703 - f1_score: 0.7129 - loss: 0.0299 - precision: 0.8256 - recall: 0.6433 - val_accuracy: 0.9341 - val_auc: 0.7575 - val_f1_score: 0.2450 - val_loss: 0.0865 - val_precision: 0.3414 - val_recall: 0.2041



Epoch 64: early stopping
Restoring model weights from the end of the best epoch: 49.
